In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, minmax_scale
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import sys 
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.utils import np_utils
import tensorflow as tf
#from keras.regularizers import L1L2
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score
from sklearn.utils import shuffle
#from keras.callbacks import EarlyStopping
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from featurewiz import featurewiz
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectKBest, f_classif, chi2, RFE
from sklearn.feature_selection import mutual_info_classif
np.random.seed(19680801)

Imported version = 0.1.70.
from featurewiz import FeatureWiz
wiz = FeatureWiz(verbose=1)
X_train_selected = wiz.fit_transform(X_train, y_train)
X_test_selected = wiz.transform(X_test)
wiz.features  ### provides a list of selected features ###
                                


In [ ]:
##prediction of offset year
def train(x_train):
    
    
  
    y_train = x_train['class']
    
    x_train = x_train.drop(['class','cluster','year','n','pct_dusted_ccn','pct_dusted_rmcl',
                           'pct_in_secbiggest_anc','rcr_low','secbiggest_anc',
                           'pct_of_biggest_anc_newish','pct_of_biggest_anc_new','n_clusts_90_anc'
                           ,'pct_in_biggest_anc','biggest_anc','pct_of_secbiggest_anc_newish'
                           ], axis = 1)
    
    x_train[['pct_of_biggest_anc','pct_is_newish','wrcr']] = minmax_scale(x_train[[
    'pct_of_biggest_anc','pct_is_newish','wrcr']])
    
    
    '''
    x_train[['wrcr','rcr_hi','rcr_mid','n_biggest_anc','n_secbiggest_anc','n_clusts_90_anc','pct_of_biggest_anc_new']] = minmax_scale(x_train[['wrcr','rcr_hi','rcr_mid','n_biggest_anc','n_secbiggest_anc','n_clusts_90_anc','pct_of_biggest_anc_new']])
    x_train = x_train.drop(['cluster','year','n','pct_is_newish','pct_dusted_ccn','pct_dusted_rmcl','biggest_anc','pct_in_biggest_anc','pct_of_biggest_anc','pct_of_biggest_anc_new','pct_of_biggest_anc_newish','secbiggest_anc'], axis = 1)#$%
    #x_train = x_train.drop(['pct_of_biggest_anc_new','pct_of_secbiggest_anc_new','animal','pct_is_newish'], axis = 1)
    x_train = x_train.drop(['pct_in_secbiggest_anc','pct_of_secbiggest_anc','pct_of_secbiggest_anc_newish','rcr_low','cited_by_clin','class'], axis = 1)
    '''
    
    y_train = np.array(y_train)
 
    model = LogisticRegression(random_state=0, C = 1).fit(x_train,y_train) ##logistioc
    
    '''crossvalidation2(x_train, y_train)
    print(model.intercept_, model.coef_, model.score(x_train, y_train))'''
    scores = cross_val_score(model, x_train, y_train, cv=20,scoring = "f1")
    print("cv score ", scores.mean())
    return model

exem = pd.read_csv("offset_prediction.csv")
exem['year_from_prize_latest'] = exem['year_from_prize_latest'].astype(int)
exem['prize_cluster_latest'] = exem['prize_cluster_latest'].astype(int)
#exem = exem[exem['exemplar_year'] < 2021]
prediction = []
for ind in exem.index:
    yr = int(exem['year_from_prize_latest'][ind])
    cl = int(exem['prize_cluster_latest'][ind])
    
    x_train = pd.read_csv("data_84.csv")
    x_test = pd.read_csv("~/Documents/PhD/project_nobel/boty/boty_outcites/features_prediction_"+str(yr)+".csv")
    x_test = x_test.dropna(axis=0)
    
    x_test = x_test[x_test['n'] > 100]
    
    print("len of xtest ",len(x_test))
       
    copy_test = x_test.copy()
    x_test = x_test.drop(['prediction','cluster','year','n',
                               'pct_in_secbiggest_anc','rcr_low','secbiggest_anc',
                               'pct_of_biggest_anc_newish','pct_of_biggest_anc_new','n_clusts_90_anc'
                               ,'pct_in_biggest_anc','biggest_anc','pct_of_secbiggest_anc_newish'
                               ], axis = 1)
    
    x_test[['pct_of_biggest_anc','pct_is_newish','wrcr']] = minmax_scale(x_test[[
    'pct_of_biggest_anc','pct_is_newish','wrcr']])
    
    
    print("hdhdhd before ",len(x_train))
    x_train = x_train[x_train['year'] != yr]
    x_train = x_train[x_train['n'] > 100]

    
    #x_train = x_train[x_train['is_research_article'] > 0.75]
    print('len of xtrain ', len(x_train))

    model = train(x_train)
    
    pred = model.predict(x_test)
    print(pred.shape)
    #print(np.sum(pred))
    print("no of positive ",np.count_nonzero(pred == 1))
    

    
    predy = model.predict_proba(x_test)
    pred_test = []
    for i in range(predy.shape[0]):
        if predy[i][0] >= predy[i][1]:
            #pred_test.append(0.00)
            pred_test.append(predy[i][1])
        else:
            pred_test.append(predy[i][1])
    
    predy = pd.DataFrame(predy, columns = ['col_0','col_1'])
    predy = predy[predy['col_1'] > 0.75]
    #print("len of df ", len(df))
    print("len of predy ", len(predy))
    copy_test['pred_log'] = pred_test
    
    a = copy_test[copy_test['cluster'] == cl]
    if len(a) == 0:
        prediction.append(-1)
        continue
    a = a.to_numpy()
    prediction.append(a[0][np.shape(a)[1] - 1])
exem['later_pred'] = prediction
exem.to_csv("offset_prediction.csv", index = False)

In [4]:
data = pd.read_csv("gold_with_pivotal.csv")
print(data.dtypes)
yy=[]
pp=[]
#data = data.head(100)
for ind in data.index:
    p = []
    y = []
    
    
    xyr = data["year"][ind] - 7
    if xyr >= 1978 and xyr <= 2021: 
        xcl = data["cl_-5"][ind]
        if np.isnan(xcl)==False:
            pred = pred_val(xyr,xcl)
            if pred != -1:
                p.append(pred)
                y.append(xyr)
        
    xyr = xyr + 1
    if xyr >= 1978 and xyr <= 2021: 
        xcl = data["cl_-4"][ind]
        if np.isnan(xcl)==False:
            pred = pred_val(xyr,xcl)
            if pred != -1:
                p.append(pred)
                y.append(xyr)
        
    xyr = xyr + 1
    if xyr >= 1978 and xyr <= 2021: 
        xcl = data["cl_-3"][ind]
        if np.isnan(xcl)==False:
            pred = pred_val(xyr,xcl)
            if pred != -1:
                p.append(pred)
                y.append(xyr)
        
    xyr = xyr + 1
    if xyr >= 1978 and xyr <= 2021: 
        xcl = data["cl_-2"][ind]
        if np.isnan(xcl)==False:
            pred = pred_val(xyr,xcl)
            if pred != -1:
                p.append(pred)
                y.append(xyr)
        
    xyr = xyr + 1
    if xyr >= 1978 and xyr <= 2021: 
        xcl = data["cl_-1"][ind]
        if np.isnan(xcl)==False:
            pred = pred_val(xyr,xcl)
            if pred != -1:
                p.append(pred)
                y.append(xyr)
    
    xyr = xyr + 1
    if xyr >= 1978 and xyr <= 2021: 
        xcl = data["cl_0"][ind]
        if np.isnan(xcl)==False:
            pred = pred_val(xyr,xcl)
            if pred != -1:
                p.append(pred)
                y.append(xyr)
        
    xyr = xyr + 1
    if xyr >= 1978 and xyr <= 2021: 
        xcl = data["cl_1"][ind]
        if np.isnan(xcl)==False:
            pred = pred_val(xyr,xcl)
            if pred != -1:
                p.append(pred)
                y.append(xyr)
        
    xyr = xyr + 1
    if xyr >= 1978 and xyr <= 2021: 
        xcl = data["cl_2"][ind]
        if np.isnan(xcl)==False:
            pred = pred_val(xyr,xcl)
            if pred != -1:
                p.append(pred)
                y.append(xyr)
    
    xyr = xyr + 1
    if xyr >= 1978 and xyr <= 2021: 
        xcl = data["cl_3"][ind]
        if np.isnan(xcl)==False:
            pred = pred_val(xyr,xcl)
            if pred != -1:
                p.append(pred)
                y.append(xyr)
        
    xyr = xyr + 1
    if xyr >= 1978 and xyr <= 2021: 
        xcl = data["cl_4"][ind]
        if np.isnan(xcl)==False:
            pred = pred_val(xyr,xcl)
            if pred != -1:
                p.append(pred)
                y.append(xyr)
        
    xyr = xyr + 1
    if xyr >= 1978 and xyr <= 2021: 
        xcl = data["cl_5"][ind]
        if np.isnan(xcl)==False:
            pred = pred_val(xyr,xcl)
            if pred != -1:
                p.append(pred)
                y.append(xyr)
    
    xyr = xyr + 1
    if xyr >= 1978 and xyr <= 2021: 
        xcl = data["cl_6"][ind]
        if np.isnan(xcl)==False:
            pred = pred_val(xyr,xcl)
            if pred != -1:
                p.append(pred)
                y.append(xyr)
        
    xyr = xyr + 1
    if xyr >= 1978 and xyr <= 2021: 
        xcl = data["cl_7"][ind]
        if np.isnan(xcl)==False:
            pred = pred_val(xyr,xcl)
            if pred != -1:
                p.append(pred)
                y.append(xyr)
        
    xyr = xyr + 1
    if xyr >= 1978 and xyr <= 2021: 
        xcl = data["cl_8"][ind]
        if np.isnan(xcl)==False:
            pred = pred_val(xyr,xcl)
            if pred != -1:
                p.append(pred)
                y.append(xyr)
        
    xyr = xyr + 1
    if xyr >= 1978 and xyr <= 2021: 
        xcl = data["cl_9"][ind]
        if np.isnan(xcl)==False:
            pred = pred_val(xyr,xcl)
            if pred != -1:
                p.append(pred)
                y.append(xyr)
        
    yy.append(y)
    pp.append(p)
data['yy'] = yy
data['pp'] = pp
data.to_csv("full_trajectory_prediction.csv", index = False)
    

pmid                   int64
year                 float64
-5                   float64
-4                   float64
-3                   float64
-2                   float64
-1                   float64
0                    float64
1                    float64
2                    float64
3                    float64
4                    float64
5                    float64
6                    float64
7                    float64
8                    float64
9                    float64
cl_-5                float64
cl_-4                float64
cl_-3                float64
cl_-2                float64
cl_-1                float64
cl_0                 float64
cl_1                 float64
cl_2                 float64
cl_3                 float64
cl_4                 float64
cl_5                 float64
cl_6                 float64
cl_7                 float64
cl_8                 float64
cl_9                 float64
exemplar_year        float64
exemplar_cl          float64
exemplar_new_r

cv score  0.7193228993228993
(11921,)
no of positive  1918
len of predy  340
len of xtest  12345
hdhdhd before  239
len of xtrain  222
cv score  0.7454523254523254
(12345,)
no of positive  1669
len of predy  300
len of xtest  12799
hdhdhd before  239
len of xtrain  226
cv score  0.7027400377400378
(12799,)
no of positive  1618
len of predy  277
len of xtest  13070
hdhdhd before  239
len of xtrain  222
cv score  0.7291869241869241
(13070,)
no of positive  1986
len of predy  349
len of xtest  13482
hdhdhd before  239
len of xtrain  228
cv score  0.728942723942724
(13482,)
no of positive  2072
len of predy  403
len of xtest  9446
hdhdhd before  239
len of xtrain  220
cv score  0.7078330003330002
(9446,)
no of positive  1564
len of predy  284
len of xtest  9768
hdhdhd before  239
len of xtrain  216
cv score  0.7010553335553336
(9768,)
no of positive  1443
len of predy  233
len of xtest  10172
hdhdhd before  239
len of xtrain  203
cv score  0.7113536463536463
(10172,)
no of positive  1393
l

cv score  0.7113536463536463
(10172,)
no of positive  1393
len of predy  232
len of xtest  10561
hdhdhd before  239
len of xtrain  215
cv score  0.7187445887445887
(10561,)
no of positive  1703
len of predy  289
len of xtest  11109
hdhdhd before  239
len of xtrain  218
cv score  0.7222793872793872
(11109,)
no of positive  1721
len of predy  245
len of xtest  11498
hdhdhd before  239
len of xtrain  213
cv score  0.7167948717948718
(11498,)
no of positive  1611
len of predy  273
len of xtest  11921
hdhdhd before  239
len of xtrain  223
cv score  0.7193228993228993
(11921,)
no of positive  1918
len of predy  340
len of xtest  12345
hdhdhd before  239
len of xtrain  222
cv score  0.7454523254523254
(12345,)
no of positive  1669
len of predy  300
len of xtest  12799
hdhdhd before  239
len of xtrain  226
cv score  0.7027400377400378
(12799,)
no of positive  1618
len of predy  277
len of xtest  13070
hdhdhd before  239
len of xtrain  222
cv score  0.7291869241869241
(13070,)
no of positive  1

cv score  0.7454523254523254
(12345,)
no of positive  1669
len of predy  300
len of xtest  12799
hdhdhd before  239
len of xtrain  226
cv score  0.7027400377400378
(12799,)
no of positive  1618
len of predy  277
len of xtest  13070
hdhdhd before  239
len of xtrain  222
cv score  0.7291869241869241
(13070,)
no of positive  1986
len of predy  349
len of xtest  13482
hdhdhd before  239
len of xtrain  228
cv score  0.728942723942724
(13482,)
no of positive  2072
len of predy  403
len of xtest  8646
hdhdhd before  239
len of xtrain  224
cv score  0.713105228105228
(8646,)
no of positive  1569
len of predy  320
len of xtest  9024
hdhdhd before  239
len of xtrain  234
cv score  0.7168475968475968
(9024,)
no of positive  1557
len of predy  282
len of xtest  9446
hdhdhd before  239
len of xtrain  220
cv score  0.7078330003330002
(9446,)
no of positive  1564
len of predy  284
len of xtest  9768
hdhdhd before  239
len of xtrain  216
cv score  0.7010553335553336
(9768,)
no of positive  1443
len of

cv score  0.7167948717948718
(11498,)
no of positive  1611
len of predy  273
len of xtest  11921
hdhdhd before  239
len of xtrain  223
cv score  0.7193228993228993
(11921,)
no of positive  1918
len of predy  340
len of xtest  12345
hdhdhd before  239
len of xtrain  222
cv score  0.7454523254523254
(12345,)
no of positive  1669
len of predy  300
len of xtest  12799
hdhdhd before  239
len of xtrain  226
cv score  0.7027400377400378
(12799,)
no of positive  1618
len of predy  277
len of xtest  13070
hdhdhd before  239
len of xtrain  222
cv score  0.7291869241869241
(13070,)
no of positive  1986
len of predy  349
len of xtest  13482
hdhdhd before  239
len of xtrain  228
cv score  0.728942723942724
(13482,)
no of positive  2072
len of predy  403
len of xtest  9024
hdhdhd before  239
len of xtrain  234
cv score  0.7168475968475968
(9024,)
no of positive  1557
len of predy  282
len of xtest  9446
hdhdhd before  239
len of xtrain  220
cv score  0.7078330003330002
(9446,)
no of positive  1564
l

...

cv score  0.7010553335553336
(9768,)
no of positive  1443
len of predy  233
len of xtest  10172
hdhdhd before  239
len of xtrain  203
cv score  0.7113536463536463
(10172,)
no of positive  1393
len of predy  232
len of xtest  10561
hdhdhd before  239
len of xtrain  215
cv score  0.7187445887445887
(10561,)
no of positive  1703
len of predy  289
len of xtest  11109
hdhdhd before  239
len of xtrain  218
cv score  0.7222793872793872
(11109,)
no of positive  1721
len of predy  245
len of xtest  11498
hdhdhd before  239
len of xtrain  213
cv score  0.7167948717948718
(11498,)
no of positive  1611
len of predy  273
len of xtest  11921
hdhdhd before  239
len of xtrain  223
cv score  0.7193228993228993
(11921,)
no of positive  1918
len of predy  340
len of xtest  12345
hdhdhd before  239
len of xtrain  222
cv score  0.7454523254523254
(12345,)
no of positive  1669
len of predy  300
len of xtest  12799
hdhdhd before  239
len of xtrain  226
cv score  0.7027400377400378
(12799,)
no of positive  16

cv score  0.7187445887445887
(10561,)
no of positive  1703
len of predy  289
len of xtest  11109
hdhdhd before  239
len of xtrain  218
cv score  0.7222793872793872
(11109,)
no of positive  1721
len of predy  245
len of xtest  11498
hdhdhd before  239
len of xtrain  213
cv score  0.7167948717948718
(11498,)
no of positive  1611
len of predy  273
len of xtest  11921
hdhdhd before  239
len of xtrain  223
cv score  0.7193228993228993
(11921,)
no of positive  1918
len of predy  340
len of xtest  12345
hdhdhd before  239
len of xtrain  222
cv score  0.7454523254523254
(12345,)
no of positive  1669
len of predy  300
len of xtest  12799
hdhdhd before  239
len of xtrain  226
cv score  0.7027400377400378
(12799,)
no of positive  1618
len of predy  277
len of xtest  13070
hdhdhd before  239
len of xtrain  222
cv score  0.7291869241869241
(13070,)
no of positive  1986
len of predy  349
len of xtest  13482
hdhdhd before  239
len of xtrain  228
cv score  0.728942723942724
(13482,)
no of positive  20

len of xtest  12345
hdhdhd before  239
len of xtrain  222
cv score  0.7454523254523254
(12345,)
no of positive  1669
len of predy  300
len of xtest  12799
hdhdhd before  239
len of xtrain  226
cv score  0.7027400377400378
(12799,)
no of positive  1618
len of predy  277
len of xtest  13070
hdhdhd before  239
len of xtrain  222
cv score  0.7291869241869241
(13070,)
no of positive  1986
len of predy  349
len of xtest  13482
hdhdhd before  239
len of xtrain  228
cv score  0.728942723942724
(13482,)
no of positive  2072
len of predy  403
len of xtest  8319
hdhdhd before  239
len of xtrain  232
cv score  0.71516317016317
(8319,)
no of positive  1619
len of predy  324
len of xtest  8646
hdhdhd before  239
len of xtrain  224
cv score  0.713105228105228
(8646,)
no of positive  1569
len of predy  320
len of xtest  9024
hdhdhd before  239
len of xtrain  234
cv score  0.7168475968475968
(9024,)
no of positive  1557
len of predy  282
len of xtest  9446
hdhdhd before  239
len of xtrain  220
cv score

cv score  0.713105228105228
(8646,)
no of positive  1569
len of predy  320
len of xtest  9024
hdhdhd before  239
len of xtrain  234
cv score  0.7168475968475968
(9024,)
no of positive  1557
len of predy  282
len of xtest  9446
hdhdhd before  239
len of xtrain  220
cv score  0.7078330003330002
(9446,)
no of positive  1564
len of predy  284
len of xtest  9768
hdhdhd before  239
len of xtrain  216
cv score  0.7010553335553336
(9768,)
no of positive  1443
len of predy  233
len of xtest  10172
hdhdhd before  239
len of xtrain  203
cv score  0.7113536463536463
(10172,)
no of positive  1393
len of predy  232
len of xtest  10561
hdhdhd before  239
len of xtrain  215
cv score  0.7187445887445887
(10561,)
no of positive  1703
len of predy  289
len of xtest  11109
hdhdhd before  239
len of xtrain  218
cv score  0.7222793872793872
(11109,)
no of positive  1721
len of predy  245
len of xtest  11498
hdhdhd before  239
len of xtrain  213
cv score  0.7167948717948718
(11498,)
no of positive  1611
len 

cv score  0.7454523254523254
(12345,)
no of positive  1669
len of predy  300
len of xtest  12799
hdhdhd before  239
len of xtrain  226
cv score  0.7027400377400378
(12799,)
no of positive  1618
len of predy  277
len of xtest  13070
hdhdhd before  239
len of xtrain  222
cv score  0.7291869241869241
(13070,)
no of positive  1986
len of predy  349
len of xtest  13482
hdhdhd before  239
len of xtrain  228
cv score  0.728942723942724
(13482,)
no of positive  2072
len of predy  403
len of xtest  9024
hdhdhd before  239
len of xtrain  234
cv score  0.7168475968475968
(9024,)
no of positive  1557
len of predy  282
len of xtest  9446
hdhdhd before  239
len of xtrain  220
cv score  0.7078330003330002
(9446,)
no of positive  1564
len of predy  284
len of xtest  9768
hdhdhd before  239
len of xtrain  216
cv score  0.7010553335553336
(9768,)
no of positive  1443
len of predy  233
len of xtest  10172
hdhdhd before  239
len of xtrain  203
cv score  0.7113536463536463
(10172,)
no of positive  1393
len

In [3]:
##prediction of offset year
def train(x_train):
    
    
  
    y_train = x_train['class']
    
    x_train = x_train.drop(['class','cluster','year','n','pct_dusted_ccn','pct_dusted_rmcl',
                           'pct_in_secbiggest_anc','rcr_low','secbiggest_anc',
                           'pct_of_biggest_anc_newish','pct_of_biggest_anc_new','n_clusts_90_anc'
                           ,'pct_in_biggest_anc','biggest_anc','pct_of_secbiggest_anc_newish'
                           ], axis = 1)
    
    x_train[['pct_of_biggest_anc','pct_is_newish','wrcr']] = minmax_scale(x_train[[
    'pct_of_biggest_anc','pct_is_newish','wrcr']])
    
    
    '''
    x_train[['wrcr','rcr_hi','rcr_mid','n_biggest_anc','n_secbiggest_anc','n_clusts_90_anc','pct_of_biggest_anc_new']] = minmax_scale(x_train[['wrcr','rcr_hi','rcr_mid','n_biggest_anc','n_secbiggest_anc','n_clusts_90_anc','pct_of_biggest_anc_new']])
    x_train = x_train.drop(['cluster','year','n','pct_is_newish','pct_dusted_ccn','pct_dusted_rmcl','biggest_anc','pct_in_biggest_anc','pct_of_biggest_anc','pct_of_biggest_anc_new','pct_of_biggest_anc_newish','secbiggest_anc'], axis = 1)#$%
    #x_train = x_train.drop(['pct_of_biggest_anc_new','pct_of_secbiggest_anc_new','animal','pct_is_newish'], axis = 1)
    x_train = x_train.drop(['pct_in_secbiggest_anc','pct_of_secbiggest_anc','pct_of_secbiggest_anc_newish','rcr_low','cited_by_clin','class'], axis = 1)
    '''
    
    y_train = np.array(y_train)
 
    model = LogisticRegression(random_state=0, C = 1).fit(x_train,y_train) ##logistioc
    
    '''crossvalidation2(x_train, y_train)
    print(model.intercept_, model.coef_, model.score(x_train, y_train))'''
    scores = cross_val_score(model, x_train, y_train, cv=20,scoring = "f1")
    print("cv score ", scores.mean())
    return model

def pred_val(yr,cl):  
    yr = int(yr)
    cl = int(cl)
    x_train = pd.read_csv("data_84.csv")
    x_test = pd.read_csv("~/Documents/PhD/project_nobel/boty/boty_dccites/features_prediction_"+str(yr)+".csv")
    x_test = x_test.dropna(axis=0)
    
    x_test = x_test[x_test['n'] > 100]
    
    print("len of xtest ",len(x_test))
       
    copy_test = x_test.copy()
    x_test = x_test.drop(['prediction','cluster','year','n',
                               'pct_in_secbiggest_anc','rcr_low','secbiggest_anc',
                               'pct_of_biggest_anc_newish','pct_of_biggest_anc_new','n_clusts_90_anc'
                               ,'pct_in_biggest_anc','biggest_anc','pct_of_secbiggest_anc_newish'
                               ], axis = 1)
    
    x_test[['pct_of_biggest_anc','pct_is_newish','wrcr']] = minmax_scale(x_test[[
    'pct_of_biggest_anc','pct_is_newish','wrcr']])
    
    
    print("hdhdhd before ",len(x_train))
    x_train = x_train[x_train['year'] != yr]
    x_train = x_train[x_train['n'] > 100]

    
    #x_train = x_train[x_train['is_research_article'] > 0.75]
    print('len of xtrain ', len(x_train))

    model = train(x_train)
    
    pred = model.predict(x_test)
    print(pred.shape)
    #print(np.sum(pred))
    print("no of positive ",np.count_nonzero(pred == 1))
    

    
    predy = model.predict_proba(x_test)
    pred_test = []
    for i in range(predy.shape[0]):
        if predy[i][0] >= predy[i][1]:
            #pred_test.append(0.00)
            pred_test.append(predy[i][1])
        else:
            pred_test.append(predy[i][1])
    
    predy = pd.DataFrame(predy, columns = ['col_0','col_1'])
    predy = predy[predy['col_1'] > 0.75]
    #print("len of df ", len(df))
    print("len of predy ", len(predy))
    copy_test['pred_log'] = pred_test
    
    a = copy_test[copy_test['cluster'] == cl]
    if len(a) == 0:
        return -1
    a = a.to_numpy()
    return a[0][np.shape(a)[1] - 1]